**For correct rendering, view this notebook in [nbviewer](https://nbviewer.org/github/markuskrecik/preference-dynamics-learning/blob/main/notebooks/10_data_generation.ipynb)**

# Synthetic Data Generation

This project solves the inverse problem for a nonlinear system of ordinary differential equations (ODEs) with neural networks. The goal in inverse problems is recovering the parameters of a system from observed data.

The system at hand describes the behavior of people through their desires and efforts ([Krecik, 2025a](https://doi.org/10.1007/s10614-025-10895-3), [Krecik, 2025b](https://www.ssrn.com/abstract=5303381)) given a set of input parameters $\mathbf{g}, \boldsymbol{\mu}, \Pi, \Gamma$:
$$
\mathbf{v}'(t) = \mathbf{g} - \Pi \cdot \mathbf{a}(t),\\
\mathbf{m}'(t) = \mathbf{u}(t) - \Gamma \cdot \mathbf{a}(t),
$$
where:
- $\mathbf{u}(t) = \max(\mathbf{v}(t), \boldsymbol{\mu})$ (desires),
- $\mathbf{a}(t) = \max(\mathbf{m}(t), \mathbf{0})$ (efforts, always ≥0).

Solving the inverse problem is generally a hard task with conventional means due to the high dimensionality of the parameter space, while neural networks are particularly well-suited for it.

**This notebook:**
1. Generates and saves a dataset of synthetic data for $n\in \{1,2,3\}$ actions
2. Gives a first glimpse at the generated data

In [ ]:
# All notebooks assume the `Notebook File Root=${workspaceFolder}` setting (in vscode), or similar
%load_ext autoreload
%autoreload 2

from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

from rich import print

from preference_dynamics.schemas import SolverConfig
from preference_dynamics.solver import create_default_sampler, generate_batch
from preference_dynamics.data import DataConfig, DataManager

## Data Generation

I generate a batch of time series samples by simulating the ODE.
`n_actions` determines the number of actions. The corresponding datasets will be saved in separate folders.
We can choose an IO handler to save in different formats.

The sampler draws random parameters and initial conditions from a (default: uniform) distribution, and checks through an Eigenvalue-based heuristic for stability of the ODE.

In [3]:
n_actions = 2  # <=3 for reasonable runtime
n_samples = 10000
seed = 42

data_dir = f"data/n{n_actions}"

sampler = create_default_sampler(n_actions=n_actions, random_seed=seed)

solver_config = SolverConfig(
    time_span=(0.0, 200.0),
    n_time_points=201,
)

print(f"Generating {n_samples} samples for n={n_actions} actions...")
batch = generate_batch(
    n_samples,
    sampler,
    solver_config,
    n_jobs=-1,
    show_progress=True,
    debug=False,
)

# DataManager handles consistent saving of raw and processed data, as well as transformations.
# Data is stored as json by default, but other IOHandlers can be chosen.
config = DataConfig(data_dir=data_dir, load_if_exists=False)
dm = DataManager(config=config)
dm.save_raw(batch)

Generating 10000 samples for n=2 actions...

Generating samples: 100%|██████████| 10000/10000 [09:25<00:00, 17.70it/s]


Let's glimpse under the hood. This is one time series sample we just generated:

In [4]:
sample = batch[0]

print(
    "Parameters and initial conditions generating the time series:\n\n",
    repr(sample.config.ode.parameters),
    "\n\n",
    repr(sample.config.ode.initial_conditions),
)
print("The created time series instance contains all information for reconstruction:\n", sample)

Parameters and initial conditions generating the time series:

 ParameterVector.from_dict({'n_actions': 2, 'g': [0.8428035290621423, 1.6280882494747453], 'mu': 
[-8.590757750252374, -1.9780301924596024], 'Pi': [[0.3161968666713354, -0.039339190198448115], 
[-0.6832656921100275, 0.6762754764490999]], 'Gamma': [[0.11601413965844697, 1.8927685707290047], 
[1.2411440630857022, 2.2141207873188633]]}) 

 ICVector.from_dict({'n_actions': 2, 'v': [2.712703466859457, -4.259553482659096], 'm': [-0.5661370858229096, 
-1.5365237618994811]})

The created time series instance contains all information for reconstruction:

TimeSeriesSample(
    sample_id='66a674a2-07b0-4652-8230-6b1de7651b5f',
    time_series=array([[ 2.71270347e+00,  3.25706258e+00,  2.90107796e+00,
         1.83047560e+00,  4.34824013e-01, -8.44470061e-01,
        -7.10039667e-01,  4.67019726e-01,  1.60612438e+00,
         2.68903792e+00,  3.72827461e+00,  4.73647742e+00,
         5.72303944e+00,  6.69457370e+00,  7.65568237e+00,
         8.60955972e+00,  9.55842158e+00,  1.05038049e+01,
         1.14467755e+01,  1.23880728e+01,  1.33282095e+01,
         1.42675414e+01,  1.52063150e+01,  1.43561722e+01,
         9.69097824e+00,  2.80894034e+00, -3.42424340e+00,
        -3.22550874e+00, -1.56765667e+00, -7.49275479e-02,
         1.25642422e+00,  2.46846969e+00,  3.59660583e+00,
         4.66636210e+00,  5.69559871e+00,  6.69672818e+00,
         7.67836283e+00,  8.64647626e+00,  9.60521175e+00,
         1.05574430e+01,  1.15051633e+01,  1.24497547e+01,
         1.33921762e+01,  1.43330926e+01,  1.52729652e+01,
         1.62121138e+01,  1.71507604e+01,  1.80890586e+01,
         1.90271153e+01,  1.99650045e+01,  2.09027774e+01,
         2.18404698e+01,  2.17329994e+01,  1.61602240e+01,
         6.58899929e+00, -3.75965154e+00, -8.59075775e+00,
        -6.60156718e+00, -4.71400970e+00, -3.08609969e+00,
        -1.66479285e+00, -3.90956324e-01,  7.79941583e-01,
         1.87934083e+00,  2.92913521e+00,  3.94452223e+00,
         4.93604516e+00,  5.91101686e+00,  6.87450911e+00,
         7.83003952e+00,  8.78004787e+00,  9.72622631e+00,
         1.06697484e+01,  1.16114282e+01,  1.25518303e+01,
         1.34913461e+01,  1.44302474e+01,  1.53687222e+01,
         1.63069014e+01,  1.72448756e+01,  1.81827075e+01,
         1.91204408e+01,  2.00581056e+01,  2.09957231e+01,
         2.19333075e+01,  2.28708693e+01,  2.38084151e+01,
         2.47459499e+01,  2.56834772e+01,  2.66209992e+01,
         2.50536883e+01,  1.67012343e+01,  4.49751348e+00,
        -7.56676196e+00, -8.59075775e+00, -8.59075775e+00,
        -6.97210287e+00, -5.24723917e+00, -3.75924145e+00,
        -2.43923746e+00, -1.23633266e+00, -1.14737218e-01,
         9.50451362e-01,  1.97651558e+00,  2.97544372e+00,
         3.95555135e+00,  4.92260576e+00,  5.88060679e+00,
         6.83232867e+00,  7.77969556e+00,  8.72404196e+00,
         9.66629345e+00,  1.06070920e+01,  1.15468828e+01,
         1.24859748e+01,  1.34245820e+01,  1.43628529e+01,
         1.53008907e+01,  1.62387667e+01,  1.71765307e+01,
         1.81142167e+01,  1.90518489e+01,  1.99894436e+01,
         2.09270124e+01,  2.18645631e+01,  2.28021014e+01,
         2.37396310e+01,  2.46771546e+01,  2.56146741e+01,
         2.65521906e+01,  2.74897051e+01,  2.84272183e+01,
         2.93647305e+01,  2.65781004e+01,  1.65310012e+01,
         2.86775917e+00, -8.59075775e+00, -8.59075775e+00,
        -8.59075775e+00, -8.28796421e+00, -6.51409323e+00,
        -4.99246687e+00, -3.64919612e+00, -2.43016319e+00,
        -1.29738314e+00, -2.24437762e-01,  8.07006451e-01,
         1.80966603e+00,  2.79236157e+00,  3.76121085e+00,
         4.72045678e+00,  5.67304209e+00,  6.62100782e+00,
         7.56576955e+00,  8.50830910e+00,  9.44930742e+00,
         1.03892369e+01,  1.13284249e+01,  1.22670987e+01,
         1.32054159e+01,  1.41434858e+01,  1.50813841e+01,
         1.60191634e+01,  1.69568602e+01,  1.78944998e+01,
         1.88320996e+01,  1.97696720e+01,  2.07072251e+01,
         2.16447652e+01,  2.25822960e+01,  2.35198204e+01,
         2.44573405e+01,  2.53948574e+01,  2.63323722e+01,
         2.72698856e+01,  2.82073979e+01,  2.91449095e+01,
         3.00824206e+01,  3.08597839e+01,  2.57760397e+01,
         1.38489472e+01, -7.08518422e-01, -8.59075775e+00,
        -8.59075775e+00, -8.59075775e+00, -8.59075775e+00,
        -6.91855742e+00, -5.41760188e+00, -4.08886871e+00,
        -2.87995100e+00, -1.75419052e+00, -6.86114602e-01,
         3.41951712e-01,  1.34226887e+00,  2.32333977e+00

## Summary

We've successfully generated synthetic time series data for the preference dynamics ODE system.

**Future extensions:**
- Add noise to the time series

**Next steps** (see `20_data_exploration.ipynb`):
- Explore the time series visually
- Clean data and explore parameter distributions